# A geographical analysis of anatomy donor registers

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json 
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

## Loading our datasets

### Anatomy donors dataset

In [ ]:
donors = pd.read_csv("out/donors.csv", index_col="id")
print(donors.shape)
donors.head()
donors.dtypes

In [ ]:
donors["death_date"] = pd.to_datetime(donors["death_date"])
donors["reception_date"] = pd.to_datetime(donors["reception_date"])
donors["return_date_sepulture"] = pd.to_datetime(donors["return_date_sepulture"])
donors["burial_date"] = pd.to_datetime(donors["burial_date"])

In [ ]:
donors.dtypes

### Geographical dataset of suburbs in NSW

**Dataset name:** _NSW Local Government Areas - Geoscape Administrative Boundaries_

This dataset was originally found on data.gov.au "NSW Suburb/Locality Boundaries - Geoscape Administrative Boundaries". Please visit the source to access the original metadata of the dataset:
https://data.gov.au/data/dataset/91e70237-d9d1-4719-a82f-e71b811154c6

**Attribution:** Incorporates or developed using Administrative Boundaries © Geoscape Australia licensed by the Commonwealth of Australia under Creative Commons Attribution 4.0 International licence (CC BY 4.0).

**Last updated:** 27/02/2024

In [ ]:
suburbs = gpd.read_file("data/nsw_localities/nsw_localities.shp")
suburbs = suburbs[["LOC_NAME", "geometry"]]
suburbs.columns = ["suburb", "geometry"]
print(suburbs.shape)
suburbs.head()

#### Checking for duplicates

In [ ]:
dup_subs = suburbs[suburbs.duplicated(["suburb"], keep=False)]
print(dup_subs.shape)
dup_subs.head()

Out of the 4615 localities in NSW, there are over 200 localities in NSW with the same name! This could present a large issue for us when we extract suburb information from the anatomy dataset and attempt to assign it a `geometry` using this dataset: our anatomy records could be assigned to a suburb with the same name in a completely different location.

Unfortunately, there is not much we can do about this currently. The reality is, bodies in the anatomy registers came from _all over New South Wales_, and so reducing the dataset in any way – such as by choosing the duplicate suburb closest to Sydney – may not be a valid step in the data cleaning process. We will just have to keep this issue of duplicates in mind when we perform the merge between our anatomy data and our localities data.

## Extracting `suburb` from donor data

We are going to run a very simple classification algorithm on the entries in our donor dataset to try and extract the suburb of where the patient passed away. 
- For each row in the donor dataset we check if `death_place` contains _any_ of the suburbs listed in the NSW Government dataset from above.
- If so, we extract **all** of the matches (as one entry could list more than one suburb in it).

In [ ]:
suburbs_ls = suburbs["suburb"].str.lower().unique()

In [ ]:
# Use regex and `extractall()` to extract suburb names from place of death
regex = r'\b(' + '|'.join(suburbs_ls) + r')\b'
matches = donors["death_place"].str.extractall(regex)
matches = matches.reset_index()
matches.columns = ["id", "match", "suburb"]

# Show a part of the dataset where one entry has multiple suburb classifications
matches[3773:3781][["id", "match", "suburb"]]

The table above shows the entry `id`, the `match` number, and the classified `suburb`. If an entry was found to contain more than one suburb, one `id` would have 2 or 3 corresponding `match` rows. Let's make this easier to see using a pivot table. 

In [ ]:
matches["suburb_count"] = matches.groupby('id').cumcount() + 1
matches = matches.pivot_table(index='id', columns='suburb_count', values='suburb', aggfunc='first').rename(columns=lambda x: f'suburb_{x}')

In [ ]:
matches.tail()

In [ ]:
matches[matches["suburb_2"].notna()].shape

### Cleaning the classifications

Now that we have a pivot table with suburb extractions, we can perform some basic cleaning before joining it back to the master dataset. For example, if all rows `suburb_1`, `suburb_2` and `suburb_3` contain the same suburb, we can just keep `suburb_1`.

In [ ]:
def find_duplicates(row: pd.Series):
    if row["suburb_1"] == row["suburb_2"]:
        return True
    
    if row["suburb_2"] == row["suburb_3"] and pd.notna(row["suburb_2"]):
        return True

    return False

# Clean duplicate entries
matches.loc[matches.apply(func=find_duplicates, axis=1), "suburb_2":] = np.nan

Now joining the two tables together:

In [ ]:
# Join pivot table on original donors dataset
donors_suburb = donors.join(matches, on="id", how="left")
donors_suburb.head(3)

Checking how many entries still contain multiple classifications:

In [ ]:
donors_suburb[donors_suburb["suburb_2"].notna() | donors_suburb["suburb_3"].notna()][["death_place", "suburb_1", "suburb_2", "suburb_3"]].head()

As you can see from the table above, there are still many rows where multiple suburbs have been extracted from the `death_place`.

One of the reasons for this is because street names often include the name of a different suburb e.g. `160 liverpool road enfield` contains two suburb names due to *Liverpool* Road running through *Enfield*.

Let's try and fix these entries. As the suburb usually comes after the street name in addresses, we will extract the last found suburb from entries containing an address. 

In [ ]:
address_keywords = ["avenue", "av", "ave", "boulevard", "bvd", "chase", "ch", "circuit", "cct", "close", "cl", "court", "ct", "crescent", "cr", "cres", "crest", "crst", "drive", "dr", "esplanade", "esp", "grange", "gra", "grove", "gr", "highway", "hwy", "lane", "la", "mall", "parade", "pde", "parkway", "pwy", "place", "pl", "road", "rd", "square", "sq", "street", "st", "terrace", "tce", "way", "walk"]
address_pattern = fr"\b(?:{'|'.join(address_keywords)})\b"

def find_and_clean_extra_suburbs(row: pd.Series):
    if ~pd.Series(row["death_place"]).str.contains(address_pattern, na=False).any():
        return row

    if pd.notna(row["suburb_3"]):
        row["suburb_1"] = row["suburb_3"]
        row["suburb_2"] = np.nan
        row["suburb_3"] = np.nan
        return row
    if pd.notna(row["suburb_2"]):
        row["suburb_1"] = row["suburb_2"]
        row["suburb_2"] = np.nan
        return row
    return row

donors_suburb = donors_suburb.apply(func=find_and_clean_extra_suburbs, axis=1)

Let's check how many entries with multiple suburb classfications still exist after completing the above cleaning.

In [ ]:
multiple_suburbs = donors_suburb[donors_suburb["suburb_2"].notna() | donors_suburb["suburb_3"].notna()][["death_place", "suburb_1", "suburb_2", "suburb_3"]]
indices_to_fix = multiple_suburbs.index
multiple_suburbs.shape

### Addressing ambiguous suburb data

There are about 100 or so entries which remain with 2 or more suburb classifications. Let's manually go through these and fix them up ourselves.

The way we do this is by first creating a data entry portal using `ipywidgets` – this way, all the data entry can be done inside this notebook, with the storage of the manually labelled suburbs taken care of by the program.

In [ ]:
output = widgets.Output()
text_input = widgets.Text(
    description='Suburb:',
    placeholder='Type correct suburb here...'
)
submit_button = widgets.Button(description="Submit")
display(text_input, submit_button, output)

i = 0
correction_code = []
def process_entry():
    global i
    with output:
        clear_output()
        if i < len(indices_to_fix):
            index_to_fix = indices_to_fix[i]

            # Save the input correction to the DataFrame
            if text_input.value:  # Check if there is any input
                # Assign the new value and print the code for this correction
                donors_suburb.loc[index_to_fix, "suburb_1"] = text_input.value
                print(f"Corrected entry:\n{donors_suburb.loc[index_to_fix]}\n")
                
                correction_code.append(f"donors_suburb.loc[{index_to_fix}, 'suburb_1'] = '{text_input.value}'")

            # Move to the next index
            i += 1

            if i < len(indices_to_fix):
                # Display the next item to correct
                index_to_fix = indices_to_fix[i]
                print(f"Next to correct: {donors_suburb.loc[index_to_fix, 'death_place']}")
                if pd.notna(donors_suburb.loc[index_to_fix, "suburb_1"]):
                    print(f"Pre-identified as 1. {donors_suburb.loc[index_to_fix, 'suburb_1']}")
                if pd.notna(donors_suburb.loc[index_to_fix, "suburb_2"]):
                    print(f"Pre-identified as 2. {donors_suburb.loc[index_to_fix, 'suburb_2']}")
                if pd.notna(donors_suburb.loc[index_to_fix, "suburb_3"]):
                    print(f"Pre-identified as 3. {donors_suburb.loc[index_to_fix, 'suburb_3']}")

                text_input.value = '' 
            else:
                print("All entries have been processed")
        else:
            print("All entries have been processed")

def on_button_clicked(b):
    process_entry()

def on_text_submit(change):
    process_entry()

submit_button.on_click(on_button_clicked)
text_input.on_submit(on_text_submit)

# Display the first entry to correct when the cell is run
with output:
    if ~indices_to_fix.empty: 
        initial_index_to_fix = indices_to_fix[0]
        print(f"First to correct: {donors_suburb.loc[initial_index_to_fix, 'death_place']}")
    else:
        print("No entries to fix")

After running the above program and entering in the correct suburb information manually, the data entry portal outputs a list of the code used to update the dataset - this way we do not have to re-enter all 100 entries everytime this notebook is run. You can see `out/fix_suburb_code.txt` for the full list of correction code.

We will now execute those lines to make sure our dataset is up to date.

In [ ]:
with open("out/fix_suburb_code.txt", "r") as f:
    for line in f:
        line = line.strip()
        exec(line)

We can now confidently select `suburb_1` for all our entries.

In [ ]:
donors_suburb = donors_suburb.drop(columns=["suburb_2", "suburb_3"])
donors_suburb = donors_suburb.rename(columns={"suburb_1": "suburb"})

In [ ]:
donors_suburb.head()

### Filling in missing suburb information

In [ ]:
missing_suburb = donors_suburb[donors_suburb["suburb"].isna()]
missing_suburb.shape

There are still about 500 entries which are missing a suburb classification. This may be due to spelling errors or the total absence of a suburb name in the place of death. Let's use a similar method as before to fill in these missing values.

In [ ]:
deathplace_suburb = {}
deathplace_missing_suburb = missing_suburb["death_place"].unique().tolist()


In [ ]:
output = widgets.Output()
text_input = widgets.Text(
    description='Suburb:',
    placeholder='Type correct suburb here...'
)
submit_button = widgets.Button(description="Submit")
display(text_input, submit_button, output)

i = 0
def process_entry():
    global i
    with output:
        clear_output()
        if i < len(deathplace_missing_suburb):
            death_place = deathplace_missing_suburb[i]

            # Save the input correction to the dictionary
            if text_input.value: 
                deathplace_suburb[death_place] = text_input.value
                print(f"'{death_place}' --> '{text_input.value}'\n")

            i += 1

            if i < len(deathplace_missing_suburb):
                print(f"Next to correct: {deathplace_missing_suburb[i]}")
                text_input.value = ''
            else:
                print("All entries have been processed")
        else:
            print("All entries have been processed")

def on_button_clicked(b):
    process_entry()

def on_text_submit(change):
    process_entry()

submit_button.on_click(on_button_clicked)
text_input.on_submit(on_text_submit)

# Display the first entry to correct when the cell is run
with output:
    if deathplace_missing_suburb:
        print(f"First to correct: {deathplace_missing_suburb[0]}")
    else:
        print("No entries to fix")

Similar to above, I have used the data entry interface above to go through and manually label the suburb for the unique places of death which were missing them. These have been saved in `out/fill_suburb_dict.txt`.

All we have to do now is to read in the dictionary and update the suburbs for the relevant entries.

In [ ]:
with open("out/fill_suburb_dict.txt", "r") as f:
    deathplace_suburb = json.load(f)

donors_suburb['suburb'] = donors_suburb['death_place'].map(deathplace_suburb).fillna(donors_suburb['suburb'])

# Optionally, filter out the entries that map to "NA" to avoid replacing them
for place, suburb in deathplace_suburb.items():
    if suburb == "NA":
        donors_suburb.loc[donors_suburb['death_place'] == place, 'suburb'] = pd.NA

Checking how many missing entries remain:

In [ ]:
missing_suburb = donors_suburb[donors_suburb["suburb"].isna()]
print(missing_suburb.shape)

In [ ]:
missing_suburb_by_cat = missing_suburb.groupby(["death_place_category"], as_index=False).size()
missing_suburb_by_cat.columns = ["death_place_category", "count"]

# Create the pie chart with labels outside the chart
fig = px.pie(missing_suburb_by_cat, values='count', names='death_place_category',
             labels={"death_place_category": "Place of death category"},
             hole=0.3)

# Update traces to position text outside and show lines
fig.update_traces(
    textposition='auto',
    textinfo='percent+label',
    pull=[0.1 for i in range(len(missing_suburb_by_cat))],
    showlegend=False,
    automargin=True
)

# Update the layout to adjust the margins
fig.update_layout(
    margin=dict(l=0, r=0, t=30, b=30)
)

fig.show()

# with open("out/missing_suburbs.pdf", "wb") as f:
#     f.write(fig.to_image(format="pdf"))

We have gone from over 470 missing entries to 86. About 2/3 of the remaining 86 entries can be attributed to deaths at unspecified morgues, and a large majority of the remaining 1/3 entries are missing place of death values as they were neonatal deaths or still births.

## Visualising the geographical distribution of place of death

### Merging `donors` and `localities`

Now it is time to merge our two datasets together. However, as we noted at the start, the `suburb` column in the `localities` dataset is NOT unique. There are multiple localities in NSW with the same name, but different locations. Keeping this in mind, let's merge the two datasets and see whether there are any such duplicate suburbs.

In [ ]:
# Convert to title case before merge
donors_suburb["suburb"] = donors_suburb["suburb"].str.title()

# Excluding entries with NA values for suburb
donors_suburb = donors_suburb.dropna(subset=["suburb"])

In [ ]:
# Merge on donors_suburb to maintain duplicates
donors_geo = gpd.GeoDataFrame(pd.merge(donors_suburb, suburbs, on="suburb", how="left"), crs="EPSG:7844")

In [ ]:
# Find all duplicates
dup_rows = donors_geo.groupby(donors_geo.columns.difference(['geometry']).tolist()).filter(lambda x: x['geometry'].nunique() > 1)

# Get all duplicate suburbs
dup_subs = dup_rows["suburb"].unique()
print(dup_subs)

The above 5 suburbs are all duplicated in our dataset. Let's see where these suburbs are on the map.

In [ ]:
fig = px.choropleth_mapbox(dup_rows,
                    geojson=dup_rows.geometry,
                    color=dup_rows.index,
                    locations=dup_rows.index,
                    center={"lat": -33.2371, "lon": 151.5623},
                    zoom=7,
                    mapbox_style="open-street-map",
                    hover_name="suburb",
                    opacity=0.6
)

fig.update_layout(
    title_text="Death Counts by Suburb (w/o outliers)",
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

After exploring the map, it is clear what is going on. Summer Hill, Punchbowl, Enmore, and Darlington are all inner city suburbs which happen to have rural counterparts! We can be sure that the entries are not from the rural suburbs by glancing over their `death_place` entries – they are all addresses of Sydney. Finally, Budgewoi is a coastal town which is naturally divided into two by the Budgewoi Lake. This is not a concern for us.

So, we will choose the entries for Summer Hill, Punchbowl, Enmore, and Darlington, which are closest to the city.

In [ ]:
from shapely.geometry import Point

sydney_centre = Point(151.2093, -33.8688)
donors_geo["dist_to_sydney"] = donors_geo["geometry"].distance(sydney_centre)

donors_geo["closest"] = True

city_suburbs = ["Summer Hill", "Punchbowl", "Enmore", "Darlington"]
furthest_entries = donors_geo[donors_geo["suburb"].isin(city_suburbs)] \
                        .sort_values(by="dist_to_sydney", ascending=False) \
                        .drop_duplicates(
                            subset=donors_geo.columns.difference(["geometry", "dist_to_sydney"]),
                            keep="first"
                        )
donors_geo.loc[furthest_entries.index, "closest"] = False
donors_geo = donors_geo[donors_geo["closest"] == True]
donors_geo = donors_geo.sort_index()
donors_geo = donors_geo.drop(columns=["dist_to_sydney", "closest"])
donors_geo["geometry"] = donors_geo["geometry"].simplify(tolerance=0.001)

In [ ]:
donors_geo

In [ ]:
suburb_counts = donors_geo.groupby(["suburb", "geometry"]).size().reset_index(name="count")
suburb_counts = gpd.GeoDataFrame(suburb_counts, geometry=suburb_counts["geometry"])
suburb_counts = suburb_counts.set_index("suburb")

In [ ]:
# Apply a logarithmic transformation to the 'count' column to reduce the impact of outliers
suburb_counts["log_count"] = np.log1p(suburb_counts["count"]) 

fig = px.choropleth_mapbox(
    suburb_counts, 
    geojson=suburb_counts.geometry,
    locations=suburb_counts.index, 
    color="log_count",
    color_continuous_scale="OrRd",
    hover_name=suburb_counts.index,
    hover_data={"log_count": False, "count": True},
    labels={"log_count": "Logarithmic death count", "count": "Death count"},
    mapbox_style="open-street-map",
    center={"lat": -33.883, "lon": 151.206},
    zoom=8,
    opacity=0.8
)

fig.update_layout(
    title_text="Death Counts by Suburb",
    margin={"r":10,"t":70,"l":20,"b":20}
)
fig.show()
# fig.write_html("out/choropleth_counts.html")

In [ ]:
most_common_cat = donors_geo.groupby(["suburb", "geometry"])["death_place_category"].agg(lambda x: pd.Series.mode(x)[0] if not x.empty else None).reset_index() # Pick the first mode
most_common_cat = gpd.GeoDataFrame(most_common_cat, geometry=most_common_cat["geometry"])

In [ ]:
fig = px.choropleth_mapbox(
    most_common_cat, 
    geojson=most_common_cat.geometry,
    locations=most_common_cat.index, 
    color="death_place_category",
    color_discrete_sequence=px.colors.qualitative.D3,
    hover_name="suburb",
    hover_data={"death_place_category": True},
    labels={"death_place_category": "Category"},
    mapbox_style="open-street-map",
    center={"lat": -33.883, "lon": 151.206},
    zoom=8,
    opacity=0.75
)

fig.update_layout(
    title_text="Suburbs by their most common place of death category",
    margin={"r":10,"t":70,"l":20,"b":20}
)
# fig.write_html("out/choropleth_categories.html")
fig.show()

In [ ]:
donors_binned = donors_geo
donors_binned["decade"] = (donors_binned["death_date"].dt.year // 10) * 10
grouped = donors_geo.groupby(["decade", "suburb", "geometry"])["death_place_category"].agg(category=(lambda x: x.mode()[0]), count=(lambda x: x.count())).reset_index()
cat_by_decade = gpd.GeoDataFrame(grouped, geometry=grouped["geometry"])

In [ ]:
fig = px.choropleth_mapbox(
    cat_by_decade,
    geojson=cat_by_decade.geometry,
    locations=cat_by_decade.index,  
    color="category",
    color_discrete_sequence=px.colors.qualitative.D3,
    hover_name="suburb",
    hover_data={"count": True, "decade": False},
    mapbox_style="open-street-map",
    center={"lat": -33.883, "lon": 151.206},
    zoom=8,
    opacity=0.75,
    animation_frame="decade"
)

fig.update_layout(
    title_text="Suburbs and their most common death place over time",
    margin={"r":10,"t":70,"l":20,"b":20}
)

# fig.write_html("out/choropleth_time.html")
fig.show()